# Lab. 9: Design of Experiments

## Introduction

#### <u>The goal of this lab is observe the effect of the DoE in the Bayesian optimization and Bio-inspired approaches.</u>

You'll have to implement four sampling methods:

Random sampling
The Halton sequence
The full factorial sampling
The Latin Hypercube sampling

- *Random sampling*
- *The Halton sequence*
- *The full factorial sampling*
- *The Latin Hypercude sampling*
---

Getting started: The following cells contain the implementation of the methods that we will use throughout this lab, together with utilities.

**NOTE**:

When studying the effect of the parameters is extremely important to vary just one parameter at a time. Therefore, it is suggested to study one parameter by fixing all the others, and then moving to the next.

Moreover, when comparing different algorithms, is very important to run each of them several times (e.g., 30) by using different initial random seeds.

In [1]:
# @title Imports
%%capture

!pip install benchmark_functions
!pip install inspyred

import random
import functools
from inspyred import ec
import numpy as np
from typing import List
from dataclasses import dataclass, field
import benchmark_functions as bf
from copy import deepcopy
import matplotlib.pyplot as plt
from math import ceil, floor
from matplotlib import pyplot as plt
import matplotlib.ticker as plticker
from inspyred import ec
from inspyred.ec import EvolutionaryComputation
from inspyred.ec import selectors, replacers, terminators
from pylab import *
from random import Random
from scipy.stats import uniform as uniform_fun
from scipy.stats import norm as norm_fun

In [2]:
#@title Helper functions

GLOBAL = 'Global'
INDIVIDUAL = 'Individual'
CORRELATED = 'Correlated'
STAR = 'star'
RING = 'ring'

class OptFun():
    def __init__(self, wf):
        self.f = wf
        self.history = []
        self.__name__ = f'OptFun({wf.__class__})'

    def __call__(self, candidates, *args, **kwargs):
        """
        Evaluate the objective function for a list of candidates.
        """
        y = []
        for x0 in candidates:
            self.history.append(deepcopy(x0))
            y.append(self.f(x0))
        return y

    def minima(self):
        return self.f.minima()

    def bounder(self):
        def fcn(candidate, *args):
            bounds = self.f.suggested_bounds()

            for i, (m, M) in enumerate(zip(*bounds)):
                if candidate[i] < m:
                    candidate[i] = m
                if candidate[i] > M:
                    candidate[i] = M
            return candidate
        return fcn

    def bounds(self):
        """
        Return the bounds of the objective function.
        """
        return self._convert_bounds(self.f.suggested_bounds())

    def heatmap(self, fn = None):
        plt.clf()
        resolution = 50
        fig = plt.figure()
        fig.canvas.manager.set_window_title('Benchmark Function: '+self.f._name)
        fig.suptitle(self.f._name)
        bounds_lower, bounds_upper = self.f.suggested_bounds()
        x = np.linspace(bounds_lower[0], bounds_upper[0], resolution)
        if self.f._n_dimensions>1:
            y = np.linspace(bounds_lower[1], bounds_upper[1], resolution)
            X, Y = np.meshgrid(x, y)
            Z = np.asarray([[self.f((X[i][j],Y[i][j])) for j in range(len(X[i]))] for i in range(len(X))])

        plt.contour(x,y,Z,15,linewidths=0.5,colors='k') # hight lines
        plt.contourf(x,y,Z,15,cmap='viridis', vmin=Z.min(), vmax=Z.max()) # heat map
        plt.xlabel('x')
        plt.ylabel('y')
        cbar = plt.colorbar()
        cbar.set_label('z')
        if len(self.history)>0:	# plot points
            xdata = [x[0] for x in self.history]
            ydata = [x[1] for x in self.history]
            plt.plot(xdata, ydata, "or-", markersize=3, linewidth=1)
        if fn is None:
            plt.show()
        else:
            plt.savefig(fn, dpi=400)

    def plot_population_evolution(self, populations, generation_step=1, single=False, grid=False, args=[]):
        plt.clf()
        resolution = 50
        fig, ax = plt.subplots(2, 3) if not single else plt.subplots(1,1)
        if single:
            ax = [ax]
            fig.set_figwidth(5)
            fig.set_figheight(5)
        else:
            ax = ax.flatten()
            fig.set_figwidth(10)
            fig.set_figheight(7)

        bounds_lower, bounds_upper = self.f.suggested_bounds()
        x = np.linspace(bounds_lower[0], bounds_upper[0], resolution)
        if self.f._n_dimensions>1:
            y = np.linspace(bounds_lower[1], bounds_upper[1], resolution)
            X, Y = np.meshgrid(x, y)
            Z = np.asarray([[self.f((X[i][j],Y[i][j])) for j in range(len(X[i]))] for i in range(len(X))])


        for i in range(min(len(populations),6)):
            if i*generation_step<len(populations):
                ax[i].contour(x,y,Z,15,linewidths=0.5,colors='k')
                ax[i].contourf(x,y,Z,15,cmap='viridis', vmin=Z.min(), vmax=Z.max()) # heat map
                ax[i].set_xlabel('x')
                ax[i].set_ylabel('y')
                ax[i].title.set_text("Generation " + str(i*generation_step))

                # scatter plot of the population
                current_pop = populations[i*generation_step]
                xdata = [x[0] for x in current_pop]
                ydata = [x[1] for x in current_pop]
                ax[i].scatter(xdata, ydata, color="r", zorder=2, label="population")

                if single:
                    break

        # Add the grid
        if single and grid:
            grid_spacing = (args["pop_init_range"][1] - args["pop_init_range"][0])/args["pop_size"]
            intervals = float(grid_spacing)
            ax[0].set_yticks(np.linspace(args["pop_init_range"][0], args["pop_init_range"][1], len(populations[0])+1), minor=False)
            ax[0].set_xticks(np.linspace(args["pop_init_range"][0], args["pop_init_range"][1], len(populations[0])+1), minor=False)
            ax[0].grid(which='both', axis='both', linestyle='-', color='k', linewidth=1.4)

        handles, labels = ax[0].get_legend_handles_labels()
        fig.legend(handles, labels, loc='upper right')
        plt.tight_layout()
        plt.show()

    def plot(self):
        plt.clf()
        values = [self.f(v) for v in self.history]
        min = func.minima()[0].score
        plt.plot(values)
        plt.axhline(min, color="r", label="optimum")
        plt.legend()
        plt.show()

    def _convert_bounds(self, bounds):
        new_bounds= []
        for i in range(len(bounds[0])):
            new_bounds.append((bounds[0][i], bounds[1][i]))
        return new_bounds

    def current_calls(self):
        return len(self.history)

def choice_without_replacement(rng, n, size) :
    result = set()
    while len(result) < size :
        result.add(rng.randint(0, n))
    return result

class NumpyRandomWrapper(RandomState):
    def __init__(self, seed=None):
        super(NumpyRandomWrapper, self).__init__(seed)

    def sample(self, population, k):
        if isinstance(population, int) :
            population = range(population)

        return asarray([population[i] for i in
                        choice_without_replacement(self, len(population), k)])
        #return #self.choice(population, k, replace=False)

    def random(self):
        return self.random_sample()

    def gauss(self, mu, sigma):
        return self.normal(mu, sigma)

def initial_pop_observer(population, num_generations, num_evaluations,
                         args):
    if num_generations == 0 :
        args["initial_pop_storage"]["individuals"] = asarray([guy.candidate
                                                 for guy in population])
        args["initial_pop_storage"]["fitnesses"] = asarray([guy.fitness
                                          for guy in population])

def generator(case, random, args):
    if case == "random":
        args["sequence"] = random_generator(random, args)
    if case == "LHS":
        args["sequence"] = lhs_generator(random, args)
    if case == "Halton":
        args["sequence"] = Halton_generator(random, args)
    if case == "FF":
        args["sequence"] = ff_generator(random, args)
    if case == "random_2d":
        args["sequence"] = random_generator_2d(random, args)
    if case == "LHS_2d":
        args["sequence"] = lhs_generator_2d(random, args)
    if case == "Halton_2d":
        args["sequence"] = Halton_generator_2d(random, args)
    if case == "FF_2d":
        args["sequence"] = ff_generator_2d(random, args)
    args["index"] = 0
    return fake_generator

def fake_generator(random, args):
    index = args["index"]
    args["index"] += 1

    return args["sequence"][index]

def generator_wrapper(func):
        @functools.wraps(func)
        def _generator(random, args):
            return np.asarray(func(random, args))
        return _generator


# helper function used to store the various populations at each generation
def my_archiver(random, population, archive, args):
    archive.append(population)
    return archive

def run_ga(random, generator_type, func, num_vars=0,
           maximize=False, **kwargs) :

    #create dictionaries to store data about initial population, and lines
    initial_pop_storage = {}

    algorithm = ec.EvolutionaryComputation(random)
    algorithm.terminator = ec.terminators.generation_termination
    algorithm.replacer = ec.replacers.generational_replacement
    algorithm.variator = [ec.variators.uniform_crossover,ec.variators.gaussian_mutation]
    algorithm.selector = ec.selectors.tournament_selection
    algorithm.archiver = my_archiver
    algorithm.observer = initial_pop_observer

    kwargs["num_selected"]=kwargs["pop_size"]
    kwargs["bounder"]=func.bounder()
    kwargs["generator"]= generator(generator_type, random, kwargs)

    final_pop = algorithm.evolve(evaluator=func,
                          maximize=False,
                          initial_pop_storage=initial_pop_storage,
                          num_vars=num_vars,
                          **kwargs)

    all_populations = algorithm.archive
    for i in range(len(all_populations)):
        all_populations[i] = [elem.candidate for elem in all_populations[i]]

    #best_guy = final_pop[0].candidate
    #best_fitness = final_pop[0].fitness
    final_pop_fitnesses = asarray([guy.fitness for guy in final_pop])
    final_pop_candidates = asarray([guy.candidate for guy in final_pop])

    sort_indexes = sorted(range(len(final_pop_fitnesses)), key=final_pop_fitnesses.__getitem__)
    final_pop_fitnesses = final_pop_fitnesses[sort_indexes]
    final_pop_candidates = final_pop_candidates[sort_indexes]

    best_guy = final_pop_candidates[0]
    best_fitness = final_pop_fitnesses[0]

    return best_guy, best_fitness, final_pop, all_populations


## Exercises

### Exercise 1/3: Implement different sampling methods

In [ ]:
def random_generator(random, args):
    pass  # TODO

def random_generator_2d(random, args):
    seq1 = random_generator(random, args)
    seq2 = random_generator(random, args)

    return np.asarray(list(zip(seq1, seq2)))

In [ ]:
def Halton_generator(random, args):
    pass  # TODO

def Halton_generator_2d(random, args):
    seq1 = Halton_generator(random, args)
    seq2 = Halton_generator(random, args)

    return np.asarray(list(zip(seq1, seq2)))

In [ ]:
def lhs_generator(random, args, j=0):
    pass  # TODO

def lhs_generator_2d(random, args):
    seq1 = lhs_generator(random, args, 0)
    seq2 = lhs_generator(random, args, 1)

    random.shuffle(seq2)

    return np.asarray(list(zip(seq1, seq2)))

In [ ]:
def ff_generator(random, args):
    pass  # TODO

def ff_generator_2d(random, args):
    pass  # TODO

### Exercise 2/3: Visualize the implemented methods

In [ ]:
# TODO

### Exercise 3/3: Genetic algorithm - testing different initial sampling methods

In [ ]:
func = OptFun(bf.Ackley(2))

args = {}
args["num_vars"] = 2 # Number of dimensions of the search space
args["gaussian_stdev"] = 1.0 # Standard deviation of the Gaussian mutations
args["tournament_size"] = 2
args["num_elites"] = 1 # number of elite individuals to maintain in each gen
args["pop_size"] = 50 # population size
args["pop_init_range"] = func.bounds()[0] # Range for the initial population
args["max_generations"] = 10 # Number of generations of the GA
args["crossover_rate"] = 0.0
args["mutation_rate"] = 1.0
args["initial_pop_size"] = args["pop_size"]

# TODO: test random, Halton, LHF and FF

## Questions

Compare the GA studied in the previous lessons with its enhanced version with different DOE techniques.

#### 1. **How do the performances increases? Are the algorithms faster to converge, or can they find better solutions?**

#### 2. **Is there an approach better than the others in terms of performance?**

#### 3. **How much do the DOEs affect the search cost?**